<a href="https://colab.research.google.com/github/Siddharth5723/AI-Powered-Regulatory-Compliance-Checker-for-Contracts/blob/main/Model_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MODEL RESEARCH:

MODULE-1:

Clause identification and risk Analysis Engine.

Tasks it needs to perform:
1. Clause extraction(NER + segmentation)

2. Clause classification

3. Risk Scoring

4. Missing clause detection

Suitable models:

1. Hugging Face Legal-BERT

2. Google AI BERT / RoBERTa



In [ ]:
!pip install transformers datasets torch scikit-learn evaluate

In [ ]:
!pip install --upgrade transformers

In [ ]:
#Hugging Face Legal-Bert
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

from sklearn.metrics import precision_score, recall_score , f1_score , accuracy_score



In [ ]:
dataset = load_dataset("lex_glue" , "ledgar")
dataset

In [ ]:
model_name = "nlpaueb/legal-bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(dataset["train"].features["label"].names)
)

In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

encoded_dataset = dataset.map(tokenize_function, batched=True)
encoded_dataset.set_format("torch")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision = precision_score(labels, predictions, average="macro")
    recall = recall_score(labels, predictions, average="macro")
    f1 = f1_score(labels, predictions, average="macro")
    acc = accuracy_score(labels, predictions)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
!pip install transformers==4.30.0

In [ ]:
!pip install --upgrade transformers

In [ ]:
training_args = TrainingArguments(
    output_dir="./legalbert_results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Step,Training Loss


In [ ]:
results = trainer.evaluate(encoded_dataset["test"])
print(results)